In [1]:
# IMPORTS
import numpy as np
import pandas as pd

import re
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

In [2]:
## HW Q1

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_filings = pd.read_html(response.text)
filings = ipo_filings[0]
filings

/tmp/ipykernel_96771/1669464139.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_filings = pd.read_html(response.text)


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
...,...,...,...,...,...
319,"Jan 21, 2020",GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000
320,"Jan 21, 2020",UTXO,"UTXO Acquisition, Inc.",$10.00,5000000
321,"Dec 9, 2019",LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000
322,"Oct 4, 2019",ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000


In [3]:
filings['Filing Date'] = pd.to_datetime(filings['Filing Date'], format='mixed')
filings[filings['Shares Offered'].astype(str).str.find('-') >= 0]

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
5,2024-04-22,DRJT,Derun Group Inc,$5.00,-
6,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,-
7,2024-04-15,GAUZ,Gauzy Ltd.,-,-
...,...,...,...,...,...
299,2021-03-02,NSAL,"Navios South American Logistics, Inc.",-,-
313,2021-01-19,DECI,"Decipher Biosciences, Inc.",-,-
315,2020-09-08,GMWX,"GameWorks, Inc.",-,-
316,2020-09-04,SCIT,Sancai Holding Group Ltd.,-,-


In [4]:
def extract_prices(input_string):
    """
    :params input_strings:
        1. replace '-' with NaN, if no price is specified
        2. return the original price, if only one number provided, e.g. $4.00
        3. return average of 2 price ranages, e.g. $8.00-$10.00
    """
    if input_string == '-': 
        return np.NaN
    else: 
        price_list = input_string.replace('$', '').split('-')
        float_price = [float(price) for price in price_list]
        return sum(float_price) / len(float_price)

filings['Avg_price'] = filings['Price Range'].apply(extract_prices)
filings

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-,NaN
...,...,...,...,...,...,...
319,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000,9.00
320,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000,10.00
321,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000,9.00
322,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000,4.00


In [5]:
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'], errors='coerce')
filings['Shares_offered_value'] = filings['Shares Offered'] * filings['Avg_price']
filings_2023Fri = filings[
    (filings['Filing Date'].dt.dayofweek == 4) & (filings['Filing Date'].dt.year == 2023)]
filings_2023Fri.shape

(32, 7)

In [6]:
### HW Q1 
filings_2023Fri['Shares_offered_value'].sum()

285700000.0

# 1) [Code Snippet 1] IPOs data from Web

* learn dataframes "stacking" (UNION ALL)
* learn about type casting
* generate new columns (simple and with function)


In [7]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

/tmp/ipykernel_96771/4213396001.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [8]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [9]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

/tmp/ipykernel_96771/1522156903.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [10]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      67 non-null     object
 1   Symbol        67 non-null     object
 2   Company Name  67 non-null     object
 3   IPO Price     67 non-null     object
 4   Current       67 non-null     object
 5   Return        67 non-null     object
dtypes: object(6)
memory usage: 3.3+ KB


In [11]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [12]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.80,7.68%


In [13]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      221 non-null    object
 1   Symbol        221 non-null    object
 2   Company Name  221 non-null    object
 3   IPO Price     221 non-null    object
 4   Current       221 non-null    object
 5   Return        221 non-null    object
dtypes: object(6)
memory usage: 10.5+ KB


In [14]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='mixed')

In [15]:
## HW Q2

ipo_tickers = stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01']['Symbol']
# ipo_tickers.drop('RYZB', inplace=True)
len(ipo_tickers)

185

In [16]:
def calculate_X(df: pd.DataFrame): 
    """
    :params df: stock price table retrieved from Yahoo Finance,
    with columns of Open, High, Low, Close, Adj Close, Volume. 
    """
    # start_price = df.head(1)['Adj Close']
    start_date = df.index.min()
    try: 
        max_30_date = start_date + pd.offsets.BDay(30)
        end_date = df.loc[start_date: max_30_date, 'Adj Close'].idxmax()
    except ValueError:
        end_date = df.loc[start_date: df.index.max(), 'Adj Close'].idxmax()

    output_df = df.reset_index()
    X = output_df[output_df['Date'] == end_date].index
    return X

In [17]:
from collections import defaultdict
max_future_growth = defaultdict(int)

for ticker in ipo_tickers: 
    if ticker == 'RYZB': 
        continue
    elif ticker == 'PTHR': 
        ticker = 'HOVR'

    price_df = yf.download(tickers = ticker,
                    period = "max",
                    interval = "1d")
    
    max_future_growth[ticker] = calculate_X(price_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [18]:
len(max_future_growth)

184

In [19]:
### HW Q2 
print('75% Quantitle: ', np.percentile(np.array(list(max_future_growth.values())), 75))

75% Quantitle:  23.0


In [48]:
np.mean(list(max_future_growth.values()))

11.880434782608695

In [49]:
np.median(list(max_future_growth.values()))

11.0

In [55]:
count_zeros = 0
for i in max_future_growth.values(): 
    if i == 0: 
        count_zeros += 1
print(count_zeros)

43


In [20]:
ipo_tickers

36     SMXT
37     VHAI
38     DYCQ
39     CHRO
40     UMAC
       ... 
216     QSG
217    CVKD
218    SKWD
219    ISRL
220    MGOL
Name: Symbol, Length: 185, dtype: object

In [21]:
price_df = yf.download(tickers='SMXT', period='max', interval='1d')
price_df.head(1)['Adj Close']

[*********************100%%**********************]  1 of 1 completed


Date
2024-02-27    8.0
Name: Adj Close, dtype: float64

In [22]:
price_df.index.min() + pd.offsets.BDay(30)

Timestamp('2024-04-09 00:00:00')

In [23]:
price_df.loc[price_df.index.min(): price_df.index.min() + pd.offsets.BDay(30), 'Adj Close'].idxmax()

Timestamp('2024-04-03 00:00:00')

In [24]:
price_df.loc[price_df.index.min(), 'Adj Close']

8.0

In [25]:
price_df.index.max()

Timestamp('2024-05-09 00:00:00')

In [26]:
## HW Q3

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']



NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

In [27]:
largest = []
for ticker in (US_STOCKS + EU_STOCKS + INDIA_STOCKS):
    price_df = yf.download(tickers = ticker,
                    period = "11y",
                    interval = "1d")
    price_df['ticker'] = ticker
    price_df['ticker_category'] = 'LARGEST'
    price_df['growth_7d'] = price_df['Adj Close'] / price_df['Adj Close'].shift(7)

    largest.append(price_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [28]:
len(largest)

33

In [29]:
large = []
for ticker in (NEW_US + NEW_EU + NEW_INDIA):
    price_df = yf.download(tickers = ticker,
                    period = "11y",
                    interval = "1d")
    price_df['ticker'] = ticker
    price_df['ticker_category'] = 'LARGE'
    price_df['growth_7d'] = price_df['Adj Close'] / price_df['Adj Close'].shift(7)

    large.append(price_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [30]:
len(large)

34

In [31]:
comparison_df = pd.concat([pd.concat(large), pd.concat(largest)])
comparison_df = comparison_df[(comparison_df.index >= pd.Timestamp('2014-01-01')) & (
    comparison_df.index < pd.Timestamp('2024-01-01')
    )]
comparison_output = comparison_df.reset_index().groupby(['Date', 'ticker_category'])['growth_7d'].aggregate('mean')
comparison_output

Date        ticker_category
2014-01-01  LARGE              1.011684
            LARGEST            1.011797
2014-01-02  LARGE              1.002958
            LARGEST            1.006333
2014-01-03  LARGE              1.000875
                                 ...   
2023-12-27  LARGEST            1.014155
2023-12-28  LARGE              0.998583
            LARGEST            1.008533
2023-12-29  LARGE              0.999313
            LARGEST            1.005337
Name: growth_7d, Length: 5190, dtype: float64

In [32]:
output_df = comparison_output.unstack(level='ticker_category')
len(output_df.index.unique())

2595

In [33]:
output_df

ticker_category,LARGE,LARGEST
Date,,
2014-01-01,1.011684,1.011797
2014-01-02,1.002958,1.006333
2014-01-03,1.000875,0.999152
2014-01-06,0.995983,0.994203
2014-01-07,0.991592,0.992509
...,...,...
2023-12-22,1.003014,1.013788
2023-12-26,1.013505,1.014900
2023-12-27,1.002167,1.014155


In [34]:
### HW Q3

large_days = len(output_df[output_df['LARGE'] > output_df['LARGEST']])
num_days = len(output_df)

print(f'Percentage of LARGE GROUP outperforms LARGEST GROUP: {large_days/num_days*100}%. ')

Percentage of LARGE GROUP outperforms LARGEST GROUP: 47.4373795761079%. 


In [35]:
## HW Q4
# 33 stocks from LARGEST STOCKS
pd.concat(largest)

,Open,High,Low,Close,Adj Close,Volume,ticker,ticker_category,growth_7d
Date,,,,,,,,,
2013-05-10,32.669998,32.720001,32.320000,32.689999,26.889845,36394900,MSFT,LARGEST,NaN
2013-05-13,32.610001,33.070000,32.549999,33.029999,27.169510,36027600,MSFT,LARGEST,NaN
2013-05-14,32.860001,33.529999,32.799999,33.529999,27.774202,56870100,MSFT,LARGEST,NaN
2013-05-15,33.450001,33.900002,33.430000,33.849998,28.039268,46303900,MSFT,LARGEST,NaN
2013-05-16,33.639999,34.150002,33.549999,34.080002,28.229799,59382900,MSFT,LARGEST,NaN
...,...,...,...,...,...,...,...,...,...
2024-05-03,3610.000000,3622.000000,3488.449951,3499.800049,3499.800049,4079696,LT.NS,LARGEST,0.969433
2024-05-06,3522.800049,3527.000000,3441.100098,3463.300049,3463.300049,2614667,LT.NS,LARGEST,0.952804
2024-05-07,3479.399902,3496.000000,3425.000000,3427.750000,3427.750000,3376013,LT.NS,LARGEST,0.938518


In [36]:
get_momentum_cii(pd.concat(largest).reset_index())

NameError: name 'get_momentum_cii' is not defined

In [ ]:
# https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab
# Update (apr 2023): Colab is now Python 3.10
# there are some other (older) ways to install talib

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4119    0  4119    0     0  15312      0 --:--:-- --:--:-- --:--:-- 15312
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0tar: libta_lib.so: Cannot create symlink to ‘libta_lib.so.0.0.0’: Permission denied
tar: libta_lib.so.0: Cannot create symlink to ‘libta_lib.so.0.0.0’: Permission denied
100  517k  100  517k    0     0   707k      0 --:--:-- --:--:-- --:--:-- 1286k
tar: libta_lib.so.0.0.0: Cannot open: Permission denied
tar: libta_lib.a: Cannot open: Permission denied
tar: Exiting with failure status due to previous errors
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4111    0  4111    0     0  19032      0 --:--:-- --:--:-- --:--:-- 19032
  0     0    0     0    0     0      0

ModuleNotFoundError: No module named 'talib'

In [37]:
import talib

def get_momentum_cii(df: pd.DataFrame) -> pd.DataFrame: 
    talib_momentum_cci = talib.CCI(df.High.values, df.Low.values, df.Close.values, timeperiod=14)
    momentum_df = pd.DataFrame({
        'Date': df.Date.values, 
        'Ticker': df.ticker, 
        'cii': talib_momentum_cci
        })
    return momentum_df

In [38]:
largest_df = pd.concat(largest)
largest_df = largest_df[(largest_df.index >= pd.Timestamp('2014-01-01')
                         ) & (largest_df.index < pd.Timestamp('2024-01-01')
    )]
largest_df_techInd = pd.DataFrame({'A': []})
gross_profit = 0
shares = 0
invest_cash = 1000

for ticker in (US_STOCKS + EU_STOCKS + INDIA_STOCKS):
    current_ticker_data = largest_df[largest_df.ticker.isin([ticker])]
    current_ticker_data['CCI'] = talib.CCI(current_ticker_data.High.values, \
                                       current_ticker_data.Low.values, current_ticker_data.Close.values, timeperiod=14)
    
    for i in range(len(current_ticker_data) - 5): 
        row_today = current_ticker_data.iloc[i]
        row_future = current_ticker_data.iloc[i + 5]

        # check if CCI > 200 and it's on Friday
        if row_today['CCI'] > 200 and row_today.name.dayofweek == 4: 
            # buy shares with $1000
            shares = invest_cash / row_today['Adj Close']
            # hold for 5 trading days and sell
            gross_profit += ((shares * row_future['Adj Close']) - invest_cash)
            shares = 0
    
            # logging ticker and profit
            print(ticker, gross_profit)

    if largest_df_techInd.empty: 
        largest_df_techInd = current_ticker_data
    else: 
        largest_df_techInd = pd.concat([largest_df_techInd, current_ticker_data], ignore_index=False)

/tmp/ipykernel_96771/2159313173.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_ticker_data['CCI'] = talib.CCI(current_ticker_data.High.values, \


MSFT 13.190642219276924
MSFT 30.398396292187044
MSFT 46.90145147683779
MSFT 80.69578826661245
MSFT 76.3457785585241
MSFT -13.144078887087744
MSFT -9.624361306440505
MSFT -41.54883758880078
MSFT -10.524757735830917
MSFT -6.587358630578251
MSFT 9.400692531928257
MSFT 17.734781362638955
MSFT 31.792443540765134
MSFT 15.359961003433341
MSFT 56.653071056855765
MSFT 41.40790697772991
MSFT 49.137566961463676
MSFT 53.21747425761919
MSFT 64.25034957845764
AAPL 100.33803421467019
AAPL 90.12914690274124
AAPL 93.65603569683367
AAPL 111.38321828203846
AAPL 119.63584700046988
AAPL 127.31029683996371
AAPL 128.59804054713663
AAPL 150.61921393405964
AAPL 198.1708280018047
AAPL 201.68848887723072
AAPL 150.56315028957442


/tmp/ipykernel_96771/2159313173.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_ticker_data['CCI'] = talib.CCI(current_ticker_data.High.values, \


AAPL 148.63238915485283
AAPL 144.24614437041782
AAPL 116.61302278589903
GOOG 106.5565679564927
GOOG 123.18441434849501
GOOG 77.73226898741291
GOOG 4.3665351310999085
GOOG 16.916454957245378
GOOG -1.5873972515278183
GOOG 15.88164315898814
GOOG 26.400481815360536
GOOG 49.767923528550114
GOOG 70.39117593938886
GOOG 83.35142317706709
GOOG 84.57519308624342
GOOG 85.74087583544929


/tmp/ipykernel_96771/2159313173.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_ticker_data['CCI'] = talib.CCI(current_ticker_data.High.values, \


...

ITC.NS 732.3421689176186
ITC.NS 747.9747602236962
ITC.NS 779.5559840717451
ITC.NS 763.5364425493069
ITC.NS 794.1323718921185
ITC.NS 894.2768547612577
ITC.NS 899.9842798501182
ITC.NS 881.5556399916246
ITC.NS 836.6988487742523
ITC.NS 835.25375580151
HINDUNILVR.NS 889.3402943913569
HINDUNILVR.NS 977.7625501593315
HINDUNILVR.NS 976.1797111347531
HINDUNILVR.NS 985.8536323329444


/tmp/ipykernel_96771/2159313173.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_ticker_data['CCI'] = talib.CCI(current_ticker_data.High.values, \


HINDUNILVR.NS 986.9669516683864
HINDUNILVR.NS 1007.8578203272527
HINDUNILVR.NS 1057.6596113702285
HINDUNILVR.NS 1043.865158885616
HINDUNILVR.NS 1066.2176038193898
HINDUNILVR.NS 1013.3280275755313
HINDUNILVR.NS 1003.9745360152333
HINDUNILVR.NS 1009.3435140043873
HINDUNILVR.NS 1004.4222914293404
HINDUNILVR.NS 1011.3026830975264
HINDUNILVR.NS 995.1733927923947
LT.NS 1070.2795692947486
LT.NS 1067.7214246681829
LT.NS 1058.0257870794267
LT.NS 1056.0120168276328
LT.NS 1090.1299507887397
LT.NS 1085.8254759981833


/tmp/ipykernel_96771/2159313173.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_ticker_data['CCI'] = talib.CCI(current_ticker_data.High.values, \


LT.NS 1072.7599022606878
LT.NS 1020.2245305381465
LT.NS 1018.6837128583743
LT.NS 1038.961593224889
LT.NS 1049.9905191554983
LT.NS 1039.4070855457796
LT.NS 1063.8827320910807
LT.NS 1066.277945400278
LT.NS 1125.1374425485465


In [39]:
largest_df_techInd = largest_df_techInd[(largest_df_techInd.index >= pd.Timestamp('2014-01-01')) & (
    largest_df_techInd.index < pd.Timestamp('2024-01-01')
    )]

gross_profit = 0
shares = 0
invest_cash = 1000

for i in range(len(largest_df_techInd) - 5): 
        row_today = largest_df_techInd.iloc[i]
        row_future = largest_df_techInd.iloc[i + 5]

        # check if CCI > 200 and it's on Friday
        if row_today['CCI'] > 200 and row_today.name.dayofweek == 4 \
            and row_future['ticker'] == row_today['ticker']: 
            # buy shares with $1000
            shares = invest_cash / row_today['Adj Close']
            # hold for 5 trading days and sell
            gross_profit += ((shares * row_future['Adj Close']) - invest_cash)
            shares = 0

            print(row_today['ticker'], gross_profit, row_future['ticker'])

MSFT 13.190642219276924 MSFT
MSFT 30.398396292187044 MSFT
MSFT 46.90145147683779 MSFT
MSFT 80.69578826661245 MSFT
MSFT 76.3457785585241 MSFT
MSFT -13.144078887087744 MSFT
MSFT -9.624361306440505 MSFT
MSFT -41.54883758880078 MSFT
MSFT -10.524757735830917 MSFT
MSFT -6.587358630578251 MSFT
MSFT 9.400692531928257 MSFT
MSFT 17.734781362638955 MSFT
MSFT 31.792443540765134 MSFT
MSFT 15.359961003433341 MSFT
MSFT 56.653071056855765 MSFT
MSFT 41.40790697772991 MSFT
MSFT 49.137566961463676 MSFT
MSFT 53.21747425761919 MSFT
MSFT 64.25034957845764 MSFT
AAPL 100.33803421467019 AAPL
AAPL 90.12914690274124 AAPL
AAPL 93.65603569683367 AAPL
AAPL 111.38321828203846 AAPL
AAPL 119.63584700046988 AAPL
AAPL 127.31029683996371 AAPL
AAPL 128.59804054713663 AAPL
AAPL 150.61921393405964 AAPL
AAPL 198.1708280018047 AAPL
AAPL 201.68848887723072 AAPL
AAPL 150.56315028957442 AAPL
AAPL 148.63238915485283 AAPL
AAPL 144.24614437041782 AAPL
AAPL 116.61302278589903 AAPL
GOOG 106.5565679564927 GOOG
GOOG 123.18441434849501 

In [40]:
### HW Q4 
gross_profit

1125.1374425485465

In [41]:
largest_done_cii = largest_df_techInd[largest_df_techInd['CCI'] > 200]
largest_done_cii.groupby(largest_done_cii.index.year).count()

,Open,High,Low,Close,Adj Close,Volume,ticker,ticker_category,growth_7d,CCI
Date,,,,,,,,,,
2014,195,195,195,195,195,195,195,195,195,195
2015,195,195,195,195,195,195,195,195,195,195
2016,192,192,192,192,192,192,192,192,192,192
2017,300,300,300,300,300,300,300,300,300,300
2018,216,216,216,216,216,216,216,216,216,216
2019,234,234,234,234,234,234,234,234,234,234
2020,223,223,223,223,223,223,223,223,223,223
2021,211,211,211,211,211,211,211,211,211,211
2022,141,141,141,141,141,141,141,141,141,141
